# **PyTorch Basic**

In [1]:
import torch
print(torch.tensor([[1,2], [3,4 ]]))
print(torch.tensor([[1,2], [3,4 ]], device='cuda:0'))

tensor([[1, 2],
        [3, 4]])
tensor([[1, 2],
        [3, 4]], device='cuda:0')


In [2]:
temp1 = torch.FloatTensor([1,2,3,4,5,6,7]) # 32 bit
temp2 = torch.DoubleTensor([1,2,3,4,5,6,7]) # 64 bit
temp3 = torch.LongTensor([1,2,3,4,5,6,7])

In [3]:
# Tensor calculation
v = torch.tensor([1,2,3])
w = torch.tensor([3,4,5])

w-v

tensor([2, 2, 2])

In [4]:
# changing tensor shape
# view function : should be contiguous

temp = torch.tensor([[1,2], [3,4]])
print(temp.shape) # original matrix

print(temp.view(4,1)) # 4 by 1 matrix
print(temp.view(-1)) # 1D array
# -1 automatically matching with other argument
print(temp.view(1, -1))
print(temp.view(-1, 1))



torch.Size([2, 2])
tensor([[1],
        [2],
        [3],
        [4]])
tensor([1, 2, 3, 4])
tensor([[1, 2, 3, 4]])
tensor([[1],
        [2],
        [3],
        [4]])


# **Custom Dataset**

In [5]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [6]:
class CustomDataset(Dataset):
  def __init__(self, csv_file):
    self.label = pd.read_csv(csv_file)
  def __len__(self):
    return len(self.label)
  def __getitem__(self, idx):
    sample = torch.tensor(self.label.iloc[idx, :3]).int()
    label = torch.tensor(self.label.iloc[idx, 3]).int()
    return sample, label

tensor_dataset = CustomDataset('sample_data/anscombe.json')
dataset = DataLoader(tensor_dataset, batch_size=4, shuffle=True)

# Mnist Dataset

In [7]:
import torchvision.transforms as transforms

mnist_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,),(1.0,))])

In [8]:
from torchvision.datasets import MNIST
import requests

# Model Definition

In [10]:
import torch.nn as nn
import torch.nn.functional as F

In [11]:
model = nn.Linear(in_features=1, out_features=1, bias=True)

In [12]:
# module inheritance

class MLP(nn.Module):
  def __init__(self, inputs):
    super(MLP, self).__init__()
    self.layer = nn.Linear(inputs, 1)
    self.activation = nn.Sigmoid()
  def forward(self, x):
    x = self.layer(x)
    x = self.activation(x)
    return x


## using Sequential DNN

In [13]:
import torch.nn as nn
class MLP(nn.Module):
  def __init__(self):
    super(MLP, self).__init__()
    self.layer1 = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5),
                                nn.ReLU(inplace=True),
                                nn.MaxPool2d(2))
    self.layer2 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=30, kernel_size=5),
                                nn.ReLU(inplace=True),
                                nn.MaxPool2d(2))
    self.layer3 = nn.Sequential(nn.Linear(in_features=30*5*5, out_features=10, bias=True),
                                nn.ReLU(inplace=True))
  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    x = x.view(x.shape[0], -1) # x.shape[0] : batch size, make 1D array
    x = self.layer3(x)
    return x
model = MLP()

# model parameter

In [17]:
import torch.optim as optim

criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
# learning scheduler : decrease learning rate when reach epoches that user setted
# initial lr is the biggest. then decreased. -> to find global minimum as fast as computer can.
scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=lambda epoch: 0.95 ** epoch)

# for epoch in range(1, 100+1):
#   for x, y in dataloader:
#     optimizer.zero_grad()
# loss = criterion(model(x), y) # Loss
# loss.backward() # backpropagation
# optimizer.step()
# scheduler.step()
